In [22]:
import pandas as pd

# load the file
pymoli = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
pymoli_df = pd.read_csv(pymoli).dropna()
pymoli_df.head().style.hide_index()

Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.530000
1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.560000
2,Ithergue48,24,Male,92,Final Critic,4.880000
3,Chamassasya86,24,Male,100,Blindscythe,3.270000
4,Iskosia90,23,Male,131,Fury,1.440000


In [23]:
# Create player demographics dataframe for later use
demographics = pymoli_df[["Gender", "SN", "Age"]].drop_duplicates()

# Create player_count variable for later use
player_count = pymoli_df["SN"].nunique()

total_players_df = pd.DataFrame({"Total Players": [player_count]}).style.hide_index()
total_players_df

Total Players
576


In [56]:
#unique values core each category
#note there are 780 purchase IDs but only 576 unique screen names(i.e. unique people that bought items)
pymoli_df.nunique()


Purchase ID    780
SN             576
Age             39
Gender           3
Item ID        179
Item Name      179
Price          145
dtype: int64

In [53]:
#Purchasing Analysis (Total)
unique_items = pymoli_df["Item ID"].nunique()
average_price = pymoli_df["Price"].mean()
number_purchases = pymoli_df["Purchase ID"].nunique()
total_revenue = pymoli_df["Price"].sum()


Values = pd.DataFrame({"Number of Unique Items": [unique_items], 
                       "Average Price": [average_price], 
                       "Number of Purchases": [number_purchases], 
                       "Total Revenue": [total_revenue]})

Values["Average Price"] = Values["Average Price"].map('${:,.2f}'.format)
Values["Total Revenue"] = Values["Total Revenue"].map('${:,.2f}'.format)

Values

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [50]:
#Gender data frame, gives you purchase count by gender
#this data frame has no duplicates 
gender = demographics["Gender"].value_counts()
gender

Male                     484
Female                    81
Other / Non-Disclosed     11
Name: Gender, dtype: int64

In [58]:
#Gender Demographics
percent_male_players = (484 / 576)*100
count_male_players = 484
percent_female_players = (81 / 576)*100
count_female_players = 81
percent_other_players = (11 / 576)*100
count_other_players = 11

#create gender values data frame
gender_values = pd.DataFrame(columns = ["Total Players", "Percentage of Players"], 
                             index = ["Male", "Female", "Other / Non-Disclosed"])

gender_values.loc["Male"] = [count_male_players, percent_male_players]
gender_values.loc["Female"] = [count_female_players, percent_female_players]
gender_values.loc["Other / Non-Disclosed"] = [count_other_players, percent_other_players]

gender_values["Percentage of Players"] = gender_values["Percentage of Players"].map('{:,.2f}%'.format)

gender_values

,Total Players,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [38]:
#Gender purchase count data frame, gives you purchase count by gender including duplicates
gender_count = pymoli_df["Gender"].value_counts()
gender_count

Male                     652
Female                   113
Other / Non-Disclosed     15
Name: Gender, dtype: int64

In [122]:
#groupby and perform calculations
gender_purchase_total = pymoli_df.groupby(["Gender"]).sum()["Price"].rename("Total Purchase Value")

gender_avg_price = pymoli_df.groupby(["Gender"]).mean()["Price"].rename("Average Purchase Price")

In [120]:
#calculations for summary table
purchase_count_male = 652
purchase_count_female = 113
purchase_count_other = 15
avg_price_male = gender_avg_price["Male"]
male_total_value = gender_purchase_total["Male"]
male_avg_total = male_total_value / count_male_players
avg_price_female = gender_avg_price["Female"]
female_total_value = gender_purchase_total["Female"]
female_avg_total = female_total_value / count_female_players
avg_price_other = gender_avg_price["Other / Non-Disclosed"]
other_total_value = gender_purchase_total["Other / Non-Disclosed"]
other_avg_total = other_total_value / count_other_players


In [133]:
# Purchasing Analysis (Gender)

purchase_summary = pd.DataFrame(columns = ["Purchase Count", "Avg Purchase Price", 
                                           "Total Purchase Value", "Avg Total Purchase per Person"], 
                             index = ["Gender", "Male", "Female", "Other / Non-Disclosed"])

#GET RID OF NAN IN GENDER ROW
purchase_summary.loc["Gender"] 
purchase_summary.loc["Male"] = [purchase_count_male, avg_price_male, male_total_value, male_avg_total]
purchase_summary.loc["Female"] = [purchase_count_female, avg_price_female, female_total_value, female_avg_total]
purchase_summary.loc["Other / Non-Disclosed"] = [purchase_count_other, avg_price_other, 
                                                 other_total_value, other_avg_total]

#format and map
purchase_summary["Avg Purchase Price"] = purchase_summary["Avg Purchase Price"].map('${:,.2f}'.format)
purchase_summary["Total Purchase Value"] = purchase_summary["Total Purchase Value"].map('${:,.2f}'.format)
purchase_summary["Avg Total Purchase per Person"] = purchase_summary["Avg Total Purchase per Person"].map('${:,.2f}'.format)

purchase_summary

,Purchase Count,Avg Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,NaN,$nan,$nan,$nan
Male,652,$3.02,"$1,967.64",$4.07
Female,113,$3.20,$361.94,$4.47
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [141]:
#Age Demographics

# Create bins for ages
bins = [0, 1, 10, 15, 20, 25, 30, 35, 40]

# Create labels for these bins
group_labels = ["0 to 9", "10 to 14", "15 to 19", "20 to 24", "25 to 29", "30 to 34",
                "35 to 39", "40+"]
    
    
# Slice the data and place it into bins
pd.cut(pymoli_df["Age"], bins, labels=group_labels).head()

0    20 to 24
1         40+
2    25 to 29
3    25 to 29
4    25 to 29
Name: Age, dtype: category
Categories (8, object): [0 to 9 < 10 to 14 < 15 to 19 < 20 to 24 < 25 to 29 < 30 to 34 < 35 to 39 < 40+]

In [140]:
pymoli_df["Age Group"] = pd.cut(pymoli_df["Age"], bins, labels=group_labels)
pymoli_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Age Group
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53,20 to 24
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56,40+
2,2,Ithergue48,24,Male,92,Final Critic,4.88,25 to 29
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27,25 to 29
4,4,Iskosia90,23,Male,131,Fury,1.44,25 to 29
